# NBA DRAFT

Testing if Git server extension works

# Imports

In [2]:
import pandas as pd

In [5]:
df = pd.read_csv("data/2024 NBA Draft Board.csv")

In [6]:
df.head()

,Player,Age,Team,League,Pos,G,MP,FG,FGA,FG%,...,AST,STL,BLK,TOV,PF,PTS,Height,Weight,Wingspan,DraftPos
0,Zaccharie Risacher,19,Bourg,LNB Élite,F,32,22.0,3.3,7.5,0.439,...,0.9,0.8,0.3,1.3,2.4,10.1,80.00,200.0,81.50,1
1,Alex Sarr,19,Perth,NBL,C,24,17.2,3.7,7.1,0.520,...,0.9,0.5,1.3,1.1,1.4,9.7,83.75,224.0,88.25,2
2,Reed Sheppard,19,Kentucky,NCAA,G,33,28.9,4.3,8.0,0.536,...,4.5,2.5,0.7,2.0,1.9,12.5,73.75,181.6,75.25,3
3,Stephon Castle,18,Connecticut,NCAA,G,34,27.0,4.0,8.5,0.472,...,2.9,0.8,0.5,1.5,2.4,11.1,77.50,210.0,81.00,4
4,Ron Holland,18,GLI,G League,F,14,33.6,6.9,15.6,0.445,...,3.0,2.5,0.6,2.9,3.3,20.6,78.50,196.8,82.75,5


In [4]:
df2 = pd.read_csv("data/2020 NBA Draft Board.csv")
df2.head()

,Player,Age,Team,League,Pos,G,MP,FG,FGA,FG%,...,AST,STL,BLK,TOV,PF,PTS,Height,Weight,Wingspan,DraftPos
0,Anthony Edwards,18,Georgia,NCAA,G,32,33.0,6.3,15.8,0.402,...,2.8,1.3,0.6,2.7,2.2,19.1,76.0,225.0,81.0,1
1,James Wiseman,18,Memphis,NCAA,C,3,23.0,6.7,8.7,0.769,...,0.3,0.3,3.0,1.0,1.7,19.7,84.0,235.0,90.0,2
2,LaMelo Ball,18,Hawks,NBL,G,12,31.3,6.3,16.7,0.375,...,6.8,1.6,0.1,2.5,2.6,17.0,79.0,180.0,82.0,3
3,Patrick Williams,18,Florida State,NCAA,F,29,22.5,3.3,7.1,0.459,...,1.0,1.0,1.0,1.7,1.6,9.2,80.0,225.0,83.0,4
4,Isaac Okoro,19,Auburn,NCAA,F,28,31.5,4.5,8.7,0.514,...,2.0,0.9,0.9,2.0,2.7,12.9,77.0,225.0,81.0,5


# References

Basketball Reference

Sports Reference

NBADraft.net